# Using Python for Research Homework: Week 4, Case Study 3

Homophily is a property of networks.  Homophily occurs when nodes that are neighbors in a network also share a characteristic more often than nodes that are not network neighbors.  In this case study, we will investigate homophily of several characteristics of individuals connected in social networks in rural India.

### Exercise 1
In Exercise 1, we will calculate the chance homophily for an arbitrary characteristic. Homophily is the proportion of edges in the network whose constituent nodes share that characteristic.

How much homophily do we expect by chance? If characteristics are distributed completely randomly, the probability that two nodes \(x\) and \(y\) share characteristic \(a\) is the probability both nodes have characteristic \(a\) , which is the marginal probability of \(a\) squared. The total probability that nodes \(x\) and \(y\) share their characteristic is therefore the sum of the square of the marginal probabilities of each characteristic in the network.

<strong>Instructions</strong>
<ul><li>Create a function <code>marginal_prob</code> that takes a dictionary <code>chars</code> with personal IDs as keys and characteristics as values; it should return a dictionary with characteristics as keys and their marginal probability (frequency of occurence of a characteristic divided by the sum of frequencies of each characteristic) as values.</li>
<li>Create a function <code>chance_homophily(chars)</code> that takes a dictionary <code>chars</code> defined as above and computes the chance homophily (homophily due to chance alone) for that characteristic.</li>
<li>A sample of three peoples' favorite colors is given in <code>favorite_colors</code>. Use your function to compute the chance homophily in this group, and store it as <code>color_homophily</code>.</li>
<li>Print <code>color_homophily</code>.</li></ul>

In [3]:
from collections import Counter
import numpy as np

def marginal_prob(chars):
    marg_prob = {}
    for ID, c in chars.items():
        marg_prob[c] = (Counter(list(chars.values()))[c])/len(chars.keys())
    return marg_prob
        
def chance_homophily(chars):
    marginal_probs = marginal_prob(chars)
    return np.sum(np.square(list(marginal_probs.values())))


favorite_colors = {"ankit":  "red", "xiaoyu": "blue", "mary":   "blue"}

color_homophily = chance_homophily(favorite_colors)
print(color_homophily)

0.5555555555555556


In [4]:
from collections import Counter

a={'a':1,'b':10,'c':20,'d':1,'e':10}
print(Counter(list(a.values())))
len(a.keys())

Counter({1: 2, 10: 2, 20: 1})


5

### Exercise 2

In the remaining exercises, we will calculate actual homophily in these village and compare the obtained values to those obtained by chance. In this exercise, we subset the data into individual villages and store them.

#### Instructions 

- `individual_characteristics.dta` contains several characteristics for each individual in the dataset such as age, religion, and caste. Use the `pandas` library to read in and store these characteristics as a dataframe called `df`.
- Store separate datasets for individuals belonging to Villages 1 and 2 as `df1` and `df2`, respectively.
- Note that some attributes may be missing for some individuals. In this case study, we will ignore rows of data where some column information is missing.
- Use the head method to display the first few entries of `df1`.

In [5]:
import pandas as pd

df  = pd.read_csv("https://courses.edx.org/asset-v1:HarvardX+PH526x+2T2019+type@asset+block@individual_characteristics.csv", low_memory=False, index_col=0)
df1 = df[df.village==1]
df2 = df[df.village==2]

df.columns

Index(['village', 'adjmatrix_key', 'pid', 'hhid', 'resp_id', 'resp_gend',
       'resp_status', 'age', 'religion', 'caste', 'subcaste', 'mothertongue',
       'speakother', 'kannada', 'tamil', 'telugu', 'hindi', 'urdu', 'english',
       'otherlang', 'educ', 'villagenative', 'native_name', 'native_type',
       'native_taluk', 'native_district', 'res_time_yrs', 'res_time_mths',
       'movereason', 'movecontact', 'movecontact_res', 'movecontact_hhid',
       'movecontact_pid', 'movecontact_name', 'workflag', 'work_freq',
       'work_freq_type', 'occupation', 'privategovt', 'work_outside',
       'work_outside_freq', 'shgparticipate', 'shg_no', 'savings',
       'savings_no', 'electioncard', 'rationcard', 'rationcard_colour'],
      dtype='object')

### Exercise 3 

In this exercise, we define a few dictionaries that enable us to look up the sex, caste, and religion of members of each village by personal ID. For Villages 1 and 2, their personal IDs are stored as `pid`.

#### Instructions 
- Define dictionaries with personal IDs as keys and a given covariate for that individual as values. Complete this for the sex, caste, and religion covariates, for Villages 1 and 2.
- For Village 1, store these dictionaries into variables named `sex1`, `caste1`, and `religion1`.
- For Village 2, store these dictionaries into variables named `sex2`, `caste2`, and `religion2`.

In [6]:
sex1      = dict(zip(df1.pid,df1.resp_gend))
caste1    = dict(zip(df1.pid,df1.caste))
religion1 = dict(zip(df1.pid,df1.religion))

sex2      = dict(zip(df2.pid,df2.resp_gend))
caste2    = dict(zip(df2.pid,df2.caste))
religion2 = dict(zip(df2.pid,df2.religion))

caste2[202802]

'OBC'

### Exercise 4

In this exercise, we will print the chance homophily of several characteristics of Villages 1 and 2. 

#### Instructions 
-  Use `chance_homophily` to compute the chance homophily for sex, caste, and religion In Villages 1 and 2. Is the chance homophily for any attribute very high for either village?

In [7]:
print(chance_homophily(sex1))
print(chance_homophily(caste1))
print(chance_homophily(religion1))

print(chance_homophily(sex2))
print(chance_homophily(caste2))
print(chance_homophily(religion2))

0.5027299861680701
0.6741488509791551
0.9804896988521925
0.5005945303210464
0.425368244800893
1.0


### Exercise 5

In this exercise, we will create a function that computes the observed homophily given a village and characteristic.

#### Instructions 
- Complete the function `homophily()`, which takes a network `G`, a dictionary of node characteristics `chars`, and node IDs `IDs`. For each node pair, determine whether a tie exists between them, as well as whether they share a characteristic. The total count of these is `num_ties` and `num_same_ties`, respectively, and their ratio is the homophily of chars in `G`. Complete the function by choosing where to increment `num_same_ties` and `num_ties`.

In [1]:
def homophily(G, chars, IDs):
    """
    Given a network G, a dict of characteristics chars for node IDs,
    and dict of node IDs for each node in the network,
    find the homophily of the network.
    """
    num_same_ties = 0
    num_ties = 0
    for n1, n2 in G.edges():
        if IDs[n1] in chars and IDs[n2] in chars:
            if G.has_edge(n1, n2):
                num_ties += 1
                # Should `num_ties` be incremented?  What about `num_same_ties`?
                if chars[IDs[n1]] == chars[IDs[n2]]:
                    num_same_ties += 1
                    # Should `num_ties` be incremented?  What about `num_same_ties`?
    return (num_same_ties / num_ties)

### Exercise 6

In this exercise, we will obtain the personal IDs for Villages 1 and 2. These will be used in the next exercise to calculate homophily for these villages.

#### Instructions 
- In this dataset, each individual has a personal ID, or PID, stored in `key_vilno_1.csv` and `key_vilno_2.csv` for villages 1 and 2, respectively. `data_filepath1` and `data_filepath2` contain the URLs to the datasets used in this exercise. Use `pd.read_csv` to read in and store `key_vilno_1.csv` and `key_vilno_2.csv` as `pid1` and `pid2` respectively. 

In [8]:
data_filepath1 = "https://courses.edx.org/asset-v1:HarvardX+PH526x+2T2019+type@asset+block@key_vilno_1.csv"
data_filepath2 = "https://courses.edx.org/asset-v1:HarvardX+PH526x+2T2019+type@asset+block@key_vilno_2.csv"

# Enter code here!
pid1=pd.read_csv(data_filepath1, index_col=0)
pid2=pd.read_csv(data_filepath2, index_col=0)
pid1

,0
0,100101
1,100102
2,100103
3,100104
4,100201
...,...
838,118202
839,118301
840,118302
841,118303


### Exercise 7

In this exercise, we will compute the homophily of several network characteristics for Villages 1 and 2 and compare them to homophily due to chance alone. The networks for these villages have been stored as networkx graph objects `G1` and `G2`.

#### Instructions 

- Use your `homophily()` function to compute the observed homophily for sex, caste, and religion in Villages 1 and 2. Print all six values.
- Use the `chance_homophily()` to compare these values to chance homophily. Are these values higher or lower than that expected by chance?

In [10]:
import networkx as nx
A1 = np.array(pd.read_csv("https://courses.edx.org/asset-v1:HarvardX+PH526x+2T2019+type@asset+block@adj_allVillageRelationships_vilno1.csv", index_col=0))
A2 = np.array(pd.read_csv("https://courses.edx.org/asset-v1:HarvardX+PH526x+2T2019+type@asset+block@adj_allVillageRelationships_vilno2.csv", index_col=0))
G1 = nx.to_networkx_graph(A1)
G2 = nx.to_networkx_graph(A2)


pid1 = pd.read_csv(data_filepath1, dtype=int)['0'].to_dict()
pid2 = pd.read_csv(data_filepath2, dtype=int)['0'].to_dict()

# Enter your code here!
print("\nhomophily()\n")
print("Village1\n")

print("Sex: ",homophily(G1, sex1, pid1))
print("Caste: ",homophily(G1, caste1, pid1))
print("Religion: ",homophily(G1, religion1, pid1))
print("\nVillage2\n")
print("Sex: ",homophily(G2, sex2, pid2))
print("Caste: ",homophily(G2, caste2, pid2))
print("Religion: ",homophily(G2, religion2, pid2))

print("\n_________________________________________________")
print("\nchance_homophily()\n")
print("Village1\n")

print("Sex: ",chance_homophily(sex1))
print("Caste: ",chance_homophily(caste1))
print("Religion: ",chance_homophily(religion1))
print("\nVillage2\n")
print("Sex: ",chance_homophily(sex2))
print("Caste: ",chance_homophily(caste2))
print("Religion: ",chance_homophily(religion2))


homophily()

Village1

Sex:  0.5908629441624366
Caste:  0.7959390862944162
Religion:  0.9908629441624366

Village2

Sex:  0.5658073270013568
Caste:  0.8276797829036635
Religion:  1.0

_________________________________________________

chance_homophily()

Village1

Sex:  0.5027299861680701
Caste:  0.6741488509791551
Religion:  0.9804896988521925

Village2

Sex:  0.5005945303210464
Caste:  0.425368244800893
Religion:  1.0
